In [1]:
using QuantumCumulants
using OrdinaryDiffEq, ModelingToolkit
using DifferentialEquations
using PyPlot
using DelimitedFiles
# using NPZ
import LinearAlgebra as la
# Define parameters
@cnumbers ω1 ω2 g γ1 γ2 Ω1 Ω2   # 1-magnon, 2-photon
h1 = FockSpace(:cavity);h2 = FockSpace(:cavity)
h=h1⊗h2
# Define the fundamental operators
@qnumbers b1::Destroy(h,1) b2::Destroy(h,2)

Ham = ω1*(b1'*b1) + ω2*(b2'*b2) +g*((b1'*b2)+(b2'*b1))+ Ω1*( b1'+b1) + Ω2*(b2'+b2)
# Collapse operators
J = [b1,b2]; rates = [2γ1,2γ2]
# Derive a set of equations
ops = [b1,b2]; eqs = meanfield(ops,Ham,J;rates=rates,order=1)


∂ₜ(⟨b1⟩) = (0 - 1im)*Ω1 + (0 - 1im)*g*⟨b2⟩ - ⟨b1⟩*γ1 + (0 - 1im)*⟨b1⟩*ω1
∂ₜ(⟨b2⟩) = (0 - 1im)*Ω2 + (0 - 1im)*g*⟨b1⟩ - ⟨b2⟩*γ2 + (0 - 1im)*⟨b2⟩*ω2


In [2]:
# Complete equations
eqs_completed = complete(eqs);
@named sys = ODESystem(eqs_completed);
A = calculate_jacobian(sys); B=[eqs_completed[1].rhs.dict[Ω1] * Ω1; eqs_completed[2].rhs.dict[Ω2] * Ω2];
Ainv=inv(A); X=Ainv*B; b1=X[1]; b2=X[2];

A

2×2 Matrix{Num}:
 -γ1 + (0-1im)*ω1         (0-1im)*g
        (0-1im)*g  -γ2 + (0-1im)*ω2

In [3]:
type = "yig_t_0.02"

root = joinpath(pwd(),"data","lone_t_sweep_yig")
# file_path = joinpath(root, "peaks_widths", "$type"*"_peaks_widths.csv")
file_path_full = joinpath(root,"$type.csv")
# df = readdlm(file_path, ',', Float64, '\n',skipstart=1)
full_data = readdlm(file_path_full,',',Float64,'\n')

# Display the first few rows of the DataFrame
frequencies = full_data[2:end,1];
s21 = full_data[2:end,2:end];
# locs = df[:,1:2] * 2e9 * pi;
# locs = sort(locs, dims=2)
Hlist = full_data[1,2:end] ./ 1000;

println(size(Hlist))

(321,)


In [17]:
#Numerical calculations of dispersion spectra for case-1 (J > Γ)
# file_path = raw"E:\Abhishek\Paper writting\paper_2_Ambt\Simulation data\QM.model.fitting.ring0 and ring 90 for 25um thickness\ring.720.920.5.csv"
files = ["yig_t_0.020.csv", "yig_t_0.033.csv", "yig_t_0.060.csv", "yig_t_0.073.csv", "yig_t_0.087.csv", "yig_t_0.100.csv"]
# files = ["yig_t_0.020.csv", "yig_t_0.033.csv", "yig_t_0.047.csv", "yig_t_0.060.csv", "yig_t_0.073.csv", "yig_t_0.087.csv", "yig_t_0.100.csv"]
nrows = 3
ncols = 2
plot_size = 3
fig, axes = subplots(nrows=nrows,ncols=ncols,figsize=(plot_size*ncols,plot_size*nrows),sharex=true,sharey=true)
# gns = 0.119:0.001:0.125
gns = [0.118,0.152,0.191, 0.215, 0.221,0.235,0.245]

for (idx,file) in enumerate(files)
    file_path = "data\\lone_t_sweep_yig\\$file"

    full_data = readdlm(file_path,',',Float64,'\n')

    # Display the first few rows of the DataFrame
    frequencies = full_data[2:end,1];
    s21 = full_data[2:end,2:end];
    Hlist = full_data[1,2:end] ./ 1000;


    γ1n=0.00014; γ2n=0.003697; 
    gn=gns[idx]
    ω2n = 5.27;
    # ω2n = 3.9790176;
    # Hlist = range(0.720, 0.920, length=100) |> collect
    occupationList1 = Float64[]; occupationList2 = Any[]
    for H in Hlist
            An=substitute( A, Dict(ω1=>1.06284384+(3.24789744)*H,ω2=>ω2n,g=>gn,γ1=>γ1n,γ2=>γ2n))
            Ann = 1im * zeros(2,2)
            for i=1:2
                for j=1:2
                    Ann[i,j] = real(An[i,j]) + 1im * imag(An[i,j])
                end
            
            end
            # if abs(H-0.8) < 1e-2
            #     println(Ann*1im)
            # end
            data0=Ann * 1im
            data1 = la.eigen(data0)
        
            datar=la.real(data1.values)
            r1n=datar[1]; r2n=datar[2];
        
        push!(occupationList1, r1n); push!(occupationList2, r2n)
    end

    ax = axes[idx]

    # println(size(s21))
    # print(size(Hlist))
    ax.pcolormesh(Hlist, frequencies, s21, cmap="inferno_r")#, levels=-7:0.1:0)
    ax.plot(Hlist,occupationList1,"w",alpha=.5);ax.plot(Hlist,occupationList2,"w",alpha=.5)
    t = Int64(parse(Float64,file[7:end-4])*1e3)
    ax.text(0.95, 5.5, "t=$t μm", color="white", fontsize=10)
    # ax.set_xlabel("Magnetic Field (kOe)"); ax.set_ylabel("Frequency (GHz)")
    # ax.set_title("$file g = $gn")

    # ax.set_xlim(0.720, 0.920)
    # ax.set_ylim(3.6, 4.4)
    ax.set_xlim(0.9,1.6)
    ax.set_ylim(5, 5.6)
end

ax_big = fig.add_subplot(111,frameon=false)  # Create a big subplot

# Turn off ax_bigis lines and ticks of the big subplot
ax_big.spines["top"].set_color("none")
ax_big.spines["bottom"].set_color("none")
ax_big.spines["left"].set_color("none")
ax_big.spines["right"].set_color("none")
ax_big.tick_params(labelcolor="none",top=false, bottom=false, left=false, right=false)

# Set common labels
ax_big.set_xlabel("H (kOe)")
ax_big.set_ylabel("Frequency (GHz)")

tight_layout()
savefig("tentative\\images\\lone_yig.png",dpi=300,bbox_inches="tight")
close(fig)
